In [70]:
# coding=utf-8
import sys
import numpy as np
import requests
import os
import pandas as pd
import csv
import sqlite3 as lite
from bs4 import BeautifulSoup
from selenium import webdriver
from shutil import copyfile
from ast import literal_eval

# Константы
all_words_links_path = 'all_words_links.npy'
vsekidki_fullwords_csv = 'vsekidki_fullwords.csv'
vsekidki_appwords_csv = "vsekidki_appwords.csv"
vsekidki_words_page = 'vsekidki_words_npy/vsekidki_words_page'
length = 45
global length

## Сделать all_words_links = [ссылки на все слова] (загрузить с диска)

In [26]:
if os.path.isfile(all_words_links_path):
    all_words_links = np.load(all_words_links_path).tolist()
else:
    vsekidki_url = "http://vsekidki.ru/word/page/"
    all_words_links = []
    for i in range(1, 45):
        r = requests.get(vsekidki_url + str(i) + '/')
        data = r.text
        soup = BeautifulSoup(data, "lxml")
        table_of_words = soup.find_all('a', class_='catalogue-heading-link')
        for link in table_of_words:
            all_words_links.append(link['href'])
    len(all_words_links)
    np.save(all_words_links_path, all_words_links)

## Основные функции

In [132]:
def Text_from_vsekidki_word(text):
    start = text.find("<")
    finish = text.find(">")
    text = text[:start] + text[finish + 1:]
    print("****11111****", text)
    check_for_tag = text.find('<', start)
    while check_for_tag != -1:
            end_of_tag = text.find('>', check_for_tag)
            text = text[:check_for_tag] + text[end_of_tag + 1:]
            check_for_tag = text.find('<', check_for_tag)
    text = text[:int(len(text)/2)]
    return text

def print_word(word):
    printing = word['name'] + "\n    ЗНАЧЕНИЕ: " + word['definition'].strip() + "\n    ПРОИСХОЖДЕНИЕ: " + word['origin'].strip() + "\n\n" 
    print(printing)
    for i in range(len(word['fulltext'])):
        print(i, word['fulltext'][i])
        
def find_quote(soup):
    quote = ""
    try:
        quote = soup.find('div', class_='quote').text
        quote = quote.replace("\'\'", "\"").replace(",,", "\"")
    except Exception as msg:
        quote = ''
    return quote

def make_origin(quote):
    origin = ""
    try:
        origin = article_body.contents[4].text
    except:
        try:
            origin = article_body.contents[4]
        except:
            origin = ''
    return (origin + quote).encode('iso-8859-1').decode('utf-8').strip()

def get_fulltext_from_article_body(article_body):
    fulltext = []
    for item in article_body.contents[2:]:
        utf_str = ""
        try:
            utf_str = item.text.strip().encode('iso-8859-1').decode('utf-8')
        except:
            try:
                utf_str = item.strip().encode('iso-8859-1').decode('utf-8')
            except:
                try:
                    utf_str = item.strip()
                except:
                    utf_str = ""
        if utf_str != "":
            fulltext.append(utf_str)
    return fulltext

def find_name(soup):
    name = ""
    try:
        name = soup.find('h1', class_='new-title').text.encode('iso-8859-1').decode('utf-8').strip()
    except:
        name = soup.find('h1', class_='new-title').text
    return name

def make_appwords_file(path):
    with open(path, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('name','definition','type','group','examples','origin','hashtags','synonyms'))
        
nthng = "_"        
        
def add_appwords_to_file(dict, path):
    with open(path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        for word in dict:
            writer.writerow((word['name'],word['definition'],"","","",word['origin'],"",nthng))
  
def make_fullwords_file(path):
    with open(path, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('name','fulltext'))


def add_fullwords_to_file(dict, path):
    with open(path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        for word in dict:
            writer.writerow((word['name'], word['fulltext']))

## Распарсить pages from 1 to length, поместить их в vsekidki_words_page(i).npy

In [189]:
for i in range(0, length):
    # to rewrite all files delete "not"
    if not os.path.isfile(vsekidki_words_page + str(i) + '.npy'):
        words = []
        print(" PAGE ", i)
        for word_link in all_words_links[(i)*30: (i + 1)*30 + 1]:
            r = requests.get(word_link)
            soup = BeautifulSoup(r.text, "lxml")
            article_body = soup.find('div', itemprop='articleBody')
            name = find_name(soup)
            definition = article_body.contents[2].encode('iso-8859-1').decode('utf-8').strip()
            quote = find_quote(soup)
            origin = make_origin(quote=quote)
            fulltext = get_fulltext_from_article_body(article_body)
            word = {"name": name,
                    "origin": origin,
                    "definition": definition,
                    "fulltext": fulltext}
            words.append(word)
        dict_path = vsekidki_words_page + str(i) + '.npy'
        np.save(dict_path, words)
print("*********\n ГОТОВО\n*********")

*********
 ГОТОВО
*********


## сделать all_words - массив всех слов

In [192]:
all_words = []
for i in range(0, length):
    read_dictionary = np.load(vsekidki_words_page + str(i) + '.npy').tolist()
    all_words += read_dictionary   
print("Слов:", len(all_words))

Слов: 1350


## Сохранить all_words в vsekidki_dict.csv

In [194]:
make_fullwords_file(vsekidki_fullwords_csv)
add_fullwords_to_file(all_words, vsekidki_fullwords_csv)

make_appwords_file(vsekidki_appwords_csv)
add_appwords_to_file(all_words, vsekidki_appwords_csv)

fullwords_df = pd.read_csv(vsekidki_fullwords_csv)
appwords_df = pd.read_csv(vsekidki_appwords_csv)

In [195]:
appwords_df.head(3)

,name,definition,type,group,examples,origin,hashtags,synonyms
0,АУЕ,А.У.Е расшифровка: Жизнь Ворам Сметь мусорам. ...,NaN,NaN,NaN,"Воряги так говорят, переводится как АРЕСТАНСКИ...",NaN,_
1,Патимейкер,Новая песня с незамысловатым текстом. Автор - ...,NaN,NaN,NaN,"Патимейкеры это те, кто организовывают тусу (д...",NaN,_
2,Аксемора,"Аксиомора ошибочное произнесение слова ""оксюмо...",NaN,NaN,NaN,"Компания ""Мегафон"" переозвучила свою рекламу с...",NaN,_


In [196]:
fullwords_df.head(3)

,name,fulltext
0,АУЕ,['А.У.Е расшифровка: Жизнь Ворам Сметь мусорам...
1,Патимейкер,['Новая песня с незамысловатым текстом. Автор ...
2,Аксемора,"['Аксиомора ошибочное произнесение слова ""оксю..."
